# Interacting particles in a box

In [1]:
import CairoMakie as cm
import LsqFit as lsq
using JLD2
using ProgressMeter
import StatsBase as sb
import Distributions as dist

## required functions

In [2]:
function init_system(; N_particles::Int64=10, L::Float64=10.0)
    # random positions from 0 to L in 3 Dimensions
    return rand(N_particles) .* L, rand(N_particles) .* L, rand(N_particles) .* L
end

function calc_distances(pos::Tuple{Float64,Float64,Float64}, particles::Tuple{Array{Float64,1},Array{Float64,1},Array{Float64,1}})
    # ToDo: add periodic boundary conditions
    x_part, y_part, z_part = particles
    x_pos, y_pos, z_pos = pos

    x_dist =  x_part .- x_pos
    y_dist =  y_part .- y_pos
    z_dist =  z_part .- z_pos

    return sqrt.(x_dist .^ 2 .+ y_dist .^ 2 .+ z_dist .^ 2), (x_dist, y_dist, z_dist)
end

function calc_distances_near(pos::Tuple{Float64,Float64,Float64}, particles::Tuple{Array{Float64,1},Array{Float64,1},Array{Float64,1}}; cutoff::Float64=1.0)

    r_dist, dist = calc_distances(pos, particles)
    x_dist, y_dist, z_dist = dist

    # remove far away particles
    x_dist = x_dist[r_dist.<=cutoff]
    y_dist = y_dist[r_dist.<=cutoff]
    z_dist = z_dist[r_dist.<=cutoff]

    r_dist = r_dist[r_dist.<=cutoff]

    # remove self-interaction
    x_dist = x_dist[r_dist.>0.0]
    y_dist = y_dist[r_dist.>0.0]
    z_dist = z_dist[r_dist.>0.0]

    r_dist = r_dist[r_dist.>0.0]

    return r_dist, (x_dist, y_dist, z_dist)
end

function calc_lj_potential(pos::Tuple{Float64,Float64,Float64}, particles::Tuple{Array{Float64,1},Array{Float64,1},Array{Float64,1}}; cutoff::Float64=1.0)
    r_dist, dist = calc_distances_near(pos, particles, cutoff=cutoff)
    return 4.0 * sum(r_dist .^ -12 .- r_dist .^ -6)
end


function calc_velocities(particle_arr::Tuple{Array{Float64,2},Array{Float64,2},Array{Float64,2}}; dt::Float64=0.01)
    x_part, y_part, z_part = particle_arr
end

function calc_lj_force(pos::Tuple{Float64,Float64,Float64}, particles::Tuple{Array{Float64,1},Array{Float64,1},Array{Float64,1}}; cutoff::Float64=1.0)
    r_dist, dist = calc_distances_near(pos, particles, cutoff=cutoff)

    x_dist, y_dist, z_dist = dist

    # calculate the force from the Lennard-Jones potential
    V_diff = -24 .* (2 .* r_dist .^ -14 .- r_dist .^ -8)

    fx = sum(V_diff .* x_dist)
    fy = sum(V_diff .* y_dist)
    fz = sum(V_diff .* z_dist)

    return fx, fy, fz
end


calc_lj_force (generic function with 1 method)

In [3]:
function verlet_step(pos::Tuple{Float64,Float64,Float64}, pos_last::Tuple{Float64,Float64,Float64}; force::Tuple{Float64,Float64,Float64}, dt::Float64=0.01)
    # calculate the new position
    return 2.0.*pos .- pos_last .+ dt^2 .* force
end

function verlet_simulate(; N_particles::Int64=10, L::Float64=10.0, dt::Float64=0.01, steps::Int64=100)
    positions = init_system(N_particles=N_particles, L=L)
    # initial velocity = 0
    positions_last = positions
    positions_arr = zeros(steps+1, N_particles, 3)
    positions_arr[1,:,1] .= positions[1]
    positions_arr[1,:,2] .= positions[2]
    positions_arr[1,:,3] .= positions[3]
    @showprogress for i in 1:steps
        x, y, z = positions
        x_last, y_last, z_last = positions_last
        x_next, y_next, z_next = zeros(N_particles), zeros(N_particles), zeros(N_particles)
        Threads.@threads for j in 1:N_particles
            pos = x[j], y[j], z[j]
            pos_last = x_last[j], y_last[j], z_last[j]
            pos_next = verlet_step(pos, pos_last, force=calc_lj_force(pos, positions, cutoff=2.5), dt=dt)
            
            # periodic boundary conditions
            x_next[j], y_next[j], z_next[j] = pos_next .% L
        end
        positions_last = positions
        positions = (x_next, y_next, z_next)
        positions_arr[i+1,:,1] .= positions[1]
        positions_arr[i+1,:,2] .= positions[2]
        positions_arr[i+1,:,3] .= positions[3]
    end
    return positions_arr
end


verlet_simulate (generic function with 1 method)

In [4]:
pos_arr = verlet_simulate(N_particles=20, steps=10_000,L=8.0, dt=0.01);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:09
